In [1]:
%pip install pycryptodome

  Obtaining dependency information for pycryptodome from https://files.pythonhosted.org/packages/1f/90/d131c0eb643290230dfa4108b7c2d135122d88b714ad241d77beb4782a76/pycryptodome-3.20.0-cp35-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 281.8 kB/s eta 0:00:07
   -- ------------------------------------- 0.1/1.8 MB 476.3 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/1.8 MB 654.6 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1.8 MB 850.1 kB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.8 MB 1.2 MB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.8 MB 1.5 MB/s eta 0:00:01
   --------------- ------------------------ 0.7/1.8 MB 1.7 MB/s eta 0:00:01
   ----------------------- ---------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

from Crypto.Cipher import AES
from Crypto.Protocol.KDF import scrypt
from Crypto.Random import get_random_bytes
import os
import hashlib
import base64

In [7]:
class PasswordManager:
    def __init__(self, master_password):
        self.master_password = master_password
        self.key = self.derive_key(master_password)
        self.file_path = "passwords.dat"

    def derive_key(self, password):
        salt = b'secure_salt'  # Salt for key derivation
        key = scrypt(password, salt, key_len=32, N=2**14, r=8, p=1)
        return key

    def encrypt(self, data):
        cipher = AES.new(self.key, AES.MODE_GCM)
        ciphertext, tag = cipher.encrypt_and_digest(data.encode('utf-8'))
        return base64.b64encode(cipher.nonce + tag + ciphertext)

    def decrypt(self, encrypted_data):
        encrypted_data = base64.b64decode(encrypted_data)
        nonce = encrypted_data[:16]
        tag = encrypted_data[16:32]
        ciphertext = encrypted_data[32:]
        cipher = AES.new(self.key, AES.MODE_GCM, nonce=nonce)
        decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)
        return decrypted_data.decode('utf-8')

    def authenticate(self, master_password):
        return self.master_password == master_password

    def save_password(self, website, username, password):
        data = f"{website},{username},{password}"
        with open(self.file_path, "a") as f:
            encrypted_data = self.encrypt(data)
            f.write(encrypted_data.decode('utf-8') + '\n')

    def retrieve_passwords(self):
        if not os.path.exists(self.file_path):
            return "No passwords saved yet."
        passwords = []
        with open(self.file_path, "r") as f:
            for line in f:
                decrypted_data = self.decrypt(line.strip())
                website, username, password = decrypted_data.split(',')
                passwords.append((website, username, password))
        return passwords

In [9]:
def main():
    master_password = input("Enter your master password: ")
    password_manager = PasswordManager(master_password)

    if not password_manager.authenticate(master_password):
        print("Authentication failed. Exiting.")
        return

    while True:
        print("\n1. Save password")
        print("2. Retrieve passwords")
        print("3. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            website = input("Enter website: ")
            username = input("Enter username: ")
            password = input("Enter password: ")
            password_manager.save_password(website, username, password)
            print("Password saved successfully.")
        elif choice == "2":
            passwords = password_manager.retrieve_passwords()
            if isinstance(passwords, str):
                print(passwords)
            else:
                print("\nSaved Passwords:")
                for i, (website, username, password) in enumerate(passwords, 1):
                    print(f"{i}. Website: {website}, Username: {username}, Password: {password}")
        elif choice == "3":
            print("Exiting.")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()



1. Save password
2. Retrieve passwords
3. Exit



Saved Passwords:
1. Website: instagram.com, Username: hamezz.b__, Password: 007123@MathanjukiJB

1. Save password
2. Retrieve passwords
3. Exit

Saved Passwords:
1. Website: instagram.com, Username: hamezz.b__, Password: 007123@MathanjukiJB

1. Save password
2. Retrieve passwords
3. Exit
Invalid choice. Please try again.

1. Save password
2. Retrieve passwords
3. Exit
Exiting.
